In [122]:
import json
import base64
import requests
import matplotlib.pyplot as plt
%matplotlib inline

In [121]:
token = "ahdps5M7sEXjBpR7cd4U9jP3yFHS7dYwVhrsv76VBC2d1veKm"
HOST = "https://smarty.mail.ru/api/v1/photo/improve"

input_file = "test.mp4"
output_file = "output.avi"

In [95]:
def request_backend(img, token):
    files = {}
    name = "file_0"
    files[name] = img
        
    meta = {
            "mode":["improve"],
            "images": [{"name": name}]
           }
                                                              
    data = {'meta': json.dumps(meta)}
    response = requests.post(HOST + "?oauth_provider=mcs&oauth_token={}" \
                      .format(token), data=data, files=files)

    return response

In [123]:
def from_base64(base64_data):
    nparr = np.fromstring(base64.b64decode(base64_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img
    #return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def parse_response(response):
    body = response.json()['body']
    improve = body['improve']
    base64_image = improve[0]['colorized_improved']
    img = from_base64(base64_image)
    return img

In [118]:
cap = cv2.VideoCapture(input_file)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter(output_file,
                      cv2.VideoWriter_fourcc('M','J','P','G'),
                      fps,
                      (frame_width, frame_height))
iFrame = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    # если цветное видео
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    
    iFrame += 1
    if frame is not None:
        buf = cv2.imencode('.jpg', frame)[1].tostring()
        response = request_backend(buf, token)
        image = parse_response(response)
        
        out.write(image)
        
        # пока, чтобы много денег не тратить!
        if iFrame > 3:
            break

cap.release()
out.release()

<VideoCapture 0x11ae6f9d0>


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """
